<a href="https://colab.research.google.com/github/davidsasso/opencv-applications/blob/main/keras_finetune_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import os, shutil

2.8.2


# Loading kaggle dataset

We have already discussed how to load kaggle datasets in the previous section. We will be using kaggle dataset on fruits. The interesting thing about this dataset is that it contains 6 classes -3 of which belong to normal fruits and 3 to rotten fruits!

NOTE: You will need to use your API token from kaggle (kaggle.json file) for downloading the kaggle dataset on colab. You can also upload the dataset manually to your google drive for doing the experiment.

In [2]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

  # The Kaggle API client expects this file to be in ~/.kaggle.
  # so move it there.
  !mkdir -p ~/.kaggle/
  !cp kaggle.json ~/.kaggle/

  # This permissions change avoids a warning on Kaggle tool startup
  !chmod 600 ~/.kaggle/kaggle.json
  !kaggle datasets download -d sriramr/apples-bananas-oranges

  # Unzip all files and suppress output
  !unzip -q apples-bananas-oranges.zip

  # Removing a duplicate of the data
  !rm -r original_data_set/original_data_set

Saving kaggle.json to kaggle (1).json
User uploaded file "kaggle.json" with length 66 bytes
apples-bananas-oranges.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# This folder contains all the class folders
main_path = 'original_data_set/'

In [6]:
# all class names
classnames = os.listdir(main_path)
print(classnames)

['rottenbanana', 'freshoranges', 'freshapples', 'freshbanana', 'rottenapples', 'rottenoranges']


In [7]:
# See how many images you have per class, this helps you set the required percentage of validation data.

for each_class in classnames:
  print("Class: {}, has {} examples".format(each_class, len(os.listdir(os.path.join(main_path, each_class)))))

Class: rottenbanana, has 306 examples
Class: freshoranges, has 206 examples
Class: freshapples, has 232 examples
Class: freshbanana, has 218 examples
Class: rottenapples, has 328 examples
Class: rottenoranges, has 222 examples


**Arrange your Data**

We have already explained in the previous section how the data should be arranged in order to feed it to training pipeline in Keras.
Here we are going to place the data neatly in train and validation folders inside a main folder called 'fruits', you can set the percentage of validation set. We have set it to 15%. So 15% of the imaged from each class will go on validation folder and the rest to traing folder.

In [ ]:
#!rm -r fruits

base_dir = 'fruits'

total_train_images = 0
total_val_images = 0

os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

# Set the percent of validation data you want.
validation_percent = 15

for each_class in classnames:
  